In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from data import data_dict, DIRECTORY
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import nltk
import torch
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing import sequence
import keras
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split


df = pd.read_csv(data_dict['news_articles'])

class BinarizedDataset(Dataset):
    def __init__(self, directory, x_col, y_col, meta_col, drop_col, merge_x):
        self.data = pd.read_csv(directory, keep_default_na=False)
        self.x_col = x_col
        self.y_col = y_col
        self.meta_col = meta_col

        self.text_data = self.data
        self.text_data = self.text_data.drop(drop_col, axis=1).drop(meta_col, axis=1)
        
        

        self.text_data[x_col] = [str(str(self.text_data[x_col][idx]) + ' ' + str(self.text_data[merge_x][idx])) for idx in range(len(self.text_data))]
        self.x_text_data = self.text_data[x_col]
        self.max_len = max([len(i) for i in self.x_text_data])

        # tool = language_tool_python.LanguageTool('en-US')
        # all_x_data = self.x_text_data.tolist()
        # all_x_data = '*$)#'.join(all_x_data)
        # all_x_data = tool.correct(all_x_data)

        # self.x_text_data = all_x_data.split('*$)#')

        self.x_text_data = self.word_vector(self.x_text_data)
        self.text_data[x_col] = [torch.LongTensor(i) for i in self.x_text_data]
        self.text_data = self.vectorize(self.text_data, [y_col])

        drop_idx = []
        for idx, entry in enumerate(self.text_data['label']):
            if entry == torch.tensor(2):
                drop_idx.append(idx)
        
        self.text_data = self.text_data.drop(df.index[[drop_idx]])
        self.text_data = self.text_data.drop([merge_x], axis=1)
        
        self.text_data = self.text_data[[x_col, y_col]]
        self.df_data = self.text_data
        self.data = self.text_data

        self.data = self.data.to_numpy()
    
    def format_text(self, token):
        clean_token = ''.join(chr for chr in token if chr.isalnum() and chr.isalpha())
        return clean_token

    def word_vector(self, data):
        x_data = data
        x_data = list(x_data)
        maximum_length = 0
        max_idx = 0
        for idx, i in enumerate(x_data):

            if len(i) > maximum_length:
                maximum_length = len(i)
                max_idx = idx
        
        t = Tokenizer()
        t.fit_on_texts(x_data)
        sequences = t.texts_to_sequences(x_data)
        sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maximum_length)
        print(x_data[0])
        print(len(x_data[0]))
        print(sequences[0])

        return sequences
    
    def vectorize(self, data_inp, columns):
        data = data_inp
        for column in columns:
            labels = list(data[column].unique())
            ref = dict(zip(data[column].unique(), [i for i in range(len(labels))]))
            print(ref)
            for idx, val in enumerate(data[column]):
                vectorized = ref[data[column][idx]]
                data[column][idx] = torch.tensor(vectorized, dtype=float)
        return data

    def __len__ (self):
        return len(self.data)

    def __getitem__ (self, idx):
        
        self.transpose_data = self.data
        self.transpose_data = self.transpose_data.transpose()
        x_data = self.transpose_data[0]
        y_data = self.transpose_data[1]

        return x_data[idx], y_data[idx]

In [3]:
print(set(list(df['type'])))
print(set(list(df['label'])))

meta_columns = ['author', 'published', 'site_url']
drop_columns = ['main_img_url', 'language', 'title', 'text', 'hasImage', 'type']

df.iloc[0]['text_without_stopwords']

{'hate', nan, 'fake', 'bias', 'state', 'junksci', 'bs', 'conspiracy', 'satire'}
{nan, 'Real', 'Fake'}


'print pay back money plus interest entire family everyone came need deported asap take two years bust go another group stealing government taxpayers group somalis stole four million government benefits months weve reported numerous cases like one muslim refugeesimmigrants commit fraud scamming systemits way control related'

In [4]:
clean_media_dataset = BinarizedDataset(data_dict['news_articles'], 'text_without_stopwords', 'label', drop_columns, meta_columns, 'title_without_stopwords')

print pay back money plus interest entire family everyone came need deported asap take two years bust go another group stealing government taxpayers group somalis stole four million government benefits months weve reported numerous cases like one muslim refugeesimmigrants commit fraud scamming systemits way control related muslims busted stole millions govt benefits
368
[   0    0    0 ...  371 5068 1104]
{'Real': 0, 'Fake': 1, '': 2}


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexes/base.py:4111: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [5]:
check = clean_media_dataset
a = list(check.df_data['label'])
if torch.tensor(2) in a:
    print("uh oh")
else:
    print('noice')

check.df_data.head()


# self.text_data[x_col] = [str(self.text_data[x_col][idx]+self.text_data[merge_x][idx]) for idx in range(len(self.text_data))]
copy_df = df
copy_df['text_without_stopwords'] = [str(copy_df['title_without_stopwords'][idx]) + ' ' + str(copy_df['text_without_stopwords'][idx]) for idx in range(len(copy_df))]

noice


In [6]:
BATCH_SIZE = 64

primary_data = clean_media_dataset #secondary option of truth_data

train_len = int(len(primary_data)*0.8)
test_len = len(primary_data) - train_len

train_set, test_set = torch.utils.data.random_split(primary_data, [train_len, test_len])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

print(len(train_set))
print(len(test_set))

num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
num_labels = np.array([train_set[i][1]for i in range(len(train_set))])

a = iter(train_loader)
b = next(a)
b = np.asarray(b)
print(b.shape)
inp_size = (b[0].shape)[1]
print(inp_size)

1676
419
(2,)
22661


<ipython-input-6-3665d531dd3a>:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
<ipython-input-6-3665d531dd3a>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/si

In [7]:
# import itertools
# ab = list(itertools.chain(*[i[0] for i in clean_media_dataset]))
# print(len(ab))
# ab = set([int(i) for i in ab])
# emb_dim = len(ab)

all_x = [i[0] for i in clean_media_dataset]
a = torch.cat(all_x)
ab = torch.unique(a) 
emb_dim = len(ab)
print(emb_dim)

47268


In [8]:
import torch.nn as nn
import torch.nn.functional as F

class RecurrentClassifier(nn.Module):
    def __init__(self, embedding_dim, input_size, hidden_size, output_size, num_layers, dropout=0.3):
        super(RecurrentClassifier, self).__init__()

        self.embedding = nn.Embedding(embedding_dim, input_size)
        self.rnn = nn.LSTM(input_size, 
                            hidden_size,
                            num_layers,
                            batch_first = True,
                            dropout=dropout)
        self.fc1 = nn.Linear(hidden_size*2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, cell) = self.rnn(x)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1, :, :]), dim=1))
        x = self.fc1(hidden)
        x = self.dropout(self.fc2(x))

        return x


max_len = len(train_set[1][0])
ref_check = 2
print(max_len)

net = RecurrentClassifier(emb_dim, int(inp_size), 50, ref_check, 2, dropout=0.2)
print(net)

22661
RecurrentClassifier(
  (embedding): Embedding(47268, 22661)
  (rnn): LSTM(22661, 50, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [9]:

LR = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=5e-3)
loss_func = torch.nn.CrossEntropyLoss()

epochs = 1000
losses = []

for step in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inp, labels = data
        inp, labels = inp.long(), labels.long()
        optimizer.zero_grad()
        outputs = net(inp)
        cost = loss_func(outputs, labels)
        cost.backward()
        optimizer.step()

        running_loss += cost.item()
    print(f'Epoch: {step}   Training Loss: {running_loss/len(train_loader)}')
print('Training Complete')  